# Setup

In [ ]:
# Setup
import os

## Data folder
def file_path_prefix(file_prefix):
    folder = f'./data/{file_prefix}'
    os.makedirs(folder, exist_ok=True)
    return f'{folder}/{file_prefix}'

# Inputs

In [ ]:
# Inputs
## Name of the area of interest
aoi_name='Salvador'

## Path to polygon file the area you want data for
aoi_url = 'https://cities-indicators.s3.eu-west-3.amazonaws.com/data/boundaries/boundary-BRA-Salvador-ADM4union.geojson'


# Get Polygon for AOI

In [ ]:
# load boundary
import geopandas as gpd

# If you are using an SSO accout, you need to be authenticated first
# !aws sso login
aoi_gdf = gpd.read_file(aoi_url, driver='GeoJSON')

aoi_gdf = aoi_gdf.to_crs(epsg=4326)

## Write to file

file_path = f'{file_path_prefix(aoi_name)}-boundary.geojson'
aoi_gdf.to_file(file_path, driver='GeoJSON')
print(f'File saved to {file_path}')



## Get area in km2 of the city rounded to the nearest integer
aoi_gdf_area = aoi_gdf['geometry'].to_crs(epsg=3857).area/ 10**6 # in km2
aoi_gdf_area = round(aoi_gdf_area.values[0])
print(f'Area: {aoi_gdf_area} sqkm')

# LULCv2

In [ ]:
from city_metrix.layers import SmartSurfaceLULC

aoi_smart_surface_lulc = SmartSurfaceLULC().get_data(aoi_gdf.total_bounds)


In [ ]:
from matplotlib.colors import ListedColormap
from enum import Enum

# Define the lulcPaletteV3
lulcPaletteV3 = [
    'b2df8a',
    'cccccc',
    '808080',
    '33a02c',
    'a6cee3',
    '000000',
    '493910',
    '5e4915',
    '735a19',
    '886a1e',
    '9d7a23',
    'b28b27',
    'f5efba',
    'c9beb6'
]

# Define the SsLulcClass Enum
class SsLulcClass(Enum):
    GREEN_SPACE_OTHER = 1
    BUILT_UP_OTHER = 2
    BARREN = 3
    OPEN_SPACE = 10
    WATER = 20
    ROADS = 30
    NON_CLASSIFIED_BUILDINGS = 40
    RESIDENTIAL_HIGH_SLOPE = 41
    NON_RESIDENTIAL_HIGH_SLOPE = 42
    UNCLASSIFIED_HIGH_SLOPE = 43
    RESIDENTIAL_LOW_SLOPE = 44
    NON_RESIDENTIAL_LOW_SLOPE = 45
    UNCLASSIFIED_LOW_SLOPE = 46
    PARKING = 50

# Create a colormap dictionary
colormap = {cls.value: f'#{color}' for cls, color in zip(SsLulcClass, lulcPaletteV3)}

# Create a ListedColormap for matplotlib
cmap = ListedColormap([colormap[key] for key in colormap.keys()])

In [ ]:
cmap

In [ ]:
# Convert hex color codes to RGBA tuples
def hex_to_rgba(hex_color):
    hex_color = hex_color.lstrip('#')
    return tuple(int(hex_color[i:i+2], 16) for i in (0, 2, 4)) + (255,)

rgba_colormap = {key: hex_to_rgba(value) for key, value in colormap.items()}

rgba_colormap

In [ ]:
rgba_colormap

In [ ]:
# Save the DataArray with the colormap as a COG
file_path = f'{file_path_prefix(aoi_name)}-smart_surface_lulc_styled.tif'

aoi_smart_surface_lulc.rio.to_raster(
    file_path,
    driver="COG",
    dtype="uint8",
    compress="deflate")

print(f'File saved to {file_path}')

In [ ]:
import rasterio

with rasterio.Env():

    with rasterio.open(file_path) as src:
        shade = src.read(1)
        meta = src.meta

    with rasterio.open(file_path, 'w', **meta) as dst:
        dst.write(shade, indexes=1)
        dst.write_colormap(
            1, rgba_colormap)
        cmap = dst.colormap(1)